In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
cd '/content/drive/Shared drives/Rahul_Mohit/ECE202A/WALG/Data/WALG_inference'

/content/drive/Shared drives/Rahul_Mohit/ECE202A/WALG/Data/WALG_inference


In [32]:
import pandas as pd
import numpy as np
import os

In [33]:
## Function to generate sliding windows.
def sliding_window(frame_length, step, X, Y):
  X_f = []
  Y_f = []
  for loop in range(0,X.shape[0]-frame_length,step):
    # X_f.append(X[loop: loop+frame_length]);
    X_f.append(
      np.vstack((
        X[loop: loop+frame_length],
        np.mean(X[loop: loop+frame_length], axis=0),
        np.std(X[loop: loop+frame_length], axis=0, dtype=np.float64)
      ))
    );
    Y_f.append(Y[loop]);
  return X_f, Y_f

## Function to store data as numpy files.
def numpify_and_store(X, y, X_name, y_name, outdatapath, shuffle=False):
      X = np.array(X)
      y = np.array(y)
      # Shuffle the train set
      if shuffle is True:
          np.random.seed(123)
          neworder = np.random.permutation(X.shape[0])
          X = X[neworder, :, :]
          y = y[neworder]
      # Save binary file
      xpath = os.path.join(outdatapath, X_name)
      ypath = os.path.join(outdatapath, y_name)
      np.save(xpath, X)
      np.save(ypath, y)
      print('Storing ' + xpath, ypath)

In [34]:
# Variables
frame_length  = int(200);
step          = 40;

csv_file_list = [
  '2020_11_22_Rahul_0_1', '2020_11_22_Mohit_0_1',
  '2020_11_22_Mohit_2_1',
  '2020_11_22_Rahul_3', '2020_11_21_Rahul_3', '2020_11_23_Rahul_3', '2020_11_24_Rahul_3', '2020_11_26_Rahul_3', '2020_11_29_Mohit_3',
  '2020_11_20_Rahul_4', '2020_11_29_Mohit_4',
  '2020_11_22_Rahul_1_1', '2020_11_22_Mohit_1_1', '2020_11_22_Rahul_1_2', '2020_11_22_Mohit_1_2',
  '2020_12_04_Mohit_up_1', '2020_12_04_Mohit_down_1', '2020_12_04_Rahul_up_1', '2020_12_04_Rahul_down_1',
  '2020_12_04_Mohit_5',
  '2020_11_30_Rahul_5'
];
exp_activities = [0,0,2,3,3,3,3,3,3,4,4,1,1,1,1,1,1,1,1,5,6];


In [35]:
for file_name in csv_file_list:
  file_path = file_name+'.csv'
  raw_data = pd.read_csv(file_path)
  start_time = raw_data.iloc[0,0]
  start_time = start_time.split()
  end_time = raw_data.iloc[-1,0]
  end_time = end_time.split(' ')
  if (np.shape(start_time)[0] == 1 or np.shape(end_time)[0] == 1 ):
    print ('Can\'t extract time from the csv file: ' + file_name);
  else:
    date_time = pd.DataFrame({
      'Date' : [start_time[0], end_time[0]],
      'Time' : [start_time[1], end_time[1]]
    })
    date_time.to_csv(file_name+'_dt.csv')
  formatted_data = pd.DataFrame(
      {
          "acc_x":   raw_data.loc[:,'accelerometerAccelerationX(G)'],
          "acc_y":   raw_data.loc[:,'accelerometerAccelerationY(G)'],
          "acc_z":   raw_data.loc[:,'accelerometerAccelerationZ(G)'],
          "gyro_x":  raw_data.loc[:,'motionRotationRateX(rad/s)'],
          "gyro_y":  raw_data.loc[:,'motionRotationRateY(rad/s)'],
          "gyro_z":  raw_data.loc[:,'motionRotationRateZ(rad/s)'],
          "label":   raw_data.loc[:,'label']
      }
  )
  x_np = formatted_data.iloc[:,0:formatted_data.shape[1]-1].to_numpy();
  y_np = formatted_data['label'].to_numpy();
  x_psw_np, y_psw_np = sliding_window(frame_length, step, x_np, y_np);
  numpify_and_store(
    x_psw_np, 
    y_psw_np, 
    X_name='X_'+file_name, 
    y_name='Y_'+file_name, 
    outdatapath='./', 
    shuffle=True 
  );

Storing ./X_2020_11_22_Rahul_0_1 ./Y_2020_11_22_Rahul_0_1
Storing ./X_2020_11_22_Mohit_0_1 ./Y_2020_11_22_Mohit_0_1
Storing ./X_2020_11_22_Mohit_2_1 ./Y_2020_11_22_Mohit_2_1
Storing ./X_2020_11_22_Rahul_3 ./Y_2020_11_22_Rahul_3
Can't extract time from the csv file: 2020_11_21_Rahul_3
Storing ./X_2020_11_21_Rahul_3 ./Y_2020_11_21_Rahul_3
Storing ./X_2020_11_23_Rahul_3 ./Y_2020_11_23_Rahul_3
Storing ./X_2020_11_24_Rahul_3 ./Y_2020_11_24_Rahul_3
Storing ./X_2020_11_26_Rahul_3 ./Y_2020_11_26_Rahul_3
Storing ./X_2020_11_29_Mohit_3 ./Y_2020_11_29_Mohit_3
Can't extract time from the csv file: 2020_11_20_Rahul_4
Storing ./X_2020_11_20_Rahul_4 ./Y_2020_11_20_Rahul_4
Storing ./X_2020_11_29_Mohit_4 ./Y_2020_11_29_Mohit_4
Storing ./X_2020_11_22_Rahul_1_1 ./Y_2020_11_22_Rahul_1_1
Storing ./X_2020_11_22_Mohit_1_1 ./Y_2020_11_22_Mohit_1_1
Storing ./X_2020_11_22_Rahul_1_2 ./Y_2020_11_22_Rahul_1_2
Storing ./X_2020_11_22_Mohit_1_2 ./Y_2020_11_22_Mohit_1_2
Storing ./X_2020_12_04_Mohit_up_1 ./Y_2020_12_04